In [ ]:
cmd_line_run = False
if not cmd_line_run:
    %matplotlib inline
collab_mode = False

if collab_mode and not cmd_line_run:
    # set up tensorflow in collab
    %tensorflow_version 2.x
# imports
import tensorflow as tf
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import numpy as np

import warnings # This ignore all the warning messages
warnings.filterwarnings('ignore')

from os import path
import os
import time

print("Tensorflow version is", tf.__version__, ", device name", tf.test.gpu_device_name())

In [ ]:
def allow_memory_growth():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

allow_memory_growth()

In [ ]:
def get_time():
    return time.strftime("%d-%m-%Y_%H-%M-%S")

In [ ]:
def process_image(img, image_shape):
    img = tf.cast(img, tf.float32)/127.5-1 # IMPORTANT, image's pixels are in the range <-1, 1>
    img = tf.image.resize(img, (64, 64))
    return img

def load_image(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img)
    return img

def display_image_from_dataset(data):
    if cmd_line_run:
        return
    for batch in data.take(1):
        image, attributes = batch
        img_ = (image[0]+1)/2
        plt.imshow(img_)
        print(img_.shape, np.min(img_), np.max(img_))

def save_generated_image(settings, epoch):
    save_dir = settings.generated_images_path
    if not path.exists(save_dir):
        os.makedirs(save_dir)
    name = path.join(save_dir,
                     'img_{}_{}.png'.format(epoch, get_time()))
    plt.savefig(name)


def show_images(images, epoch, settings, save_images=False, display_images=False):
    print("image pixels range", np.min(images), np.max(images), "std", np.std(images))
    num_of_images = min(10, images.shape[0])
    # (x, y=1)
    plt.figure(figsize=(num_of_images, 1))
    for i in range(num_of_images):
        plt.subplot(1, num_of_images, i + 1)
        img = images[i, :, :, :].numpy() #
        img = (img * 127.5 + 127.5).astype(np.uint8)
        plt.imshow(img)
        plt.axis('off')
    

    if save_images:
        save_generated_image(settings, epoch)
    if display_images and not cmd_line_run:
        plt.show()

def load_dataset(dataset_path, image_shape, use_manually_downloaded_dataset=True, preprocess_images=True, shuffle_size=500, seed=101):
    if use_manually_downloaded_dataset:
        img_path = path.join(dataset_path, '*.jpg')
        data = tf.data.Dataset.list_files(img_path, seed=seed)\
                              .shuffle(shuffle_size)\
                              .map(load_image)
        # for each image return a tuple (image, attributes), in this case we have no attributes
        if preprocess_images:
            data = data.map(lambda x: (process_image(x, image_shape), dict()))
        else:
            data = data.map(lambda x: (x, dict()))
            
    else:
        dataset_name = 'celeb_a'
        data = tfds.load(dataset_name, split=tfds.Split.TRAIN)\
                   .shuffle(shuffle_size)
        # for each image return a tuple (image, attributes), ignore 'landmarks'
        if preprocess_images:
            data = data\
                .map(lambda x: (process_image(x['image'], image_shape), x['attributes']))
        else:
            data = data\
                .map(lambda x: (x['image'], x['attributes']))
    return data

In [ ]:
class Settings:
    def __init__(self, collab_mode):
        self.root_local_path = os.getcwd()
        self.root_gdrive_path = '/content/drive'
        self.gdrive_project_path = 'My Drive/pp/GSN/FaceGenerator'
        self.dataset_name = "celeb_a"
        self.subdataset_dir="1000"
        self.dataset_image_size = (28, 28)
        self.image_size = (64, 64)
        self.image_channels = 3
        self.generator_input_shape = (self.image_size[0], self.image_channels)
        self.gdrive_mounted = False
        self.collab_mode = collab_mode
        self.batch_size = 10
        self.epochs = 100
        self.save_models = False # save models at the end?
        self.mount_gdrive()
        
    @property
    def run_name(self):
        return 'run_{}'.format(self.subdataset_dir)
        # return "{}_epochs_{}_batch_{}".format(self.epochs, self.batch_size, self.subdataset_dir)
    
    @property
    def image_shape(self):
        return (*self.image_size, self.image_channels)

    @property
    def download_path(self):
        return path.join('.', 'datasets', self.dataset_name)
    # def download_path(self):
    #     return path.join(self.get_base_path, 'datasets', self.dataset_name)

    @property
    def dataset_path(self):
        return path.join(self.download_path, self.subdataset_dir)

    @property
    def tensorboard_log_dir(self):
        return path.join(self.get_base_path, 'saved_state', self.run_name, 'tensorboard_logs')

    @property
    def checkpoint_dir(self):
        return path.join(self.get_base_path, 'saved_state', self.run_name, "ckpt")

    @property
    def model_save_path(self):
        return path.join(self.get_base_path, 'saved_state', self.run_name, 'models')
    
    @property
    def generated_images_path(self):
        return path.join(self.get_base_path, 'saved_state', self.run_name, 'generated_images')
    
    @property
    def get_base_path(self):
        if self.collab_mode:
            return path.join(self.root_gdrive_path, self.gdrive_project_path)
        else:
            return self.root_local_path

    def mount_gdrive(self):
        if self.collab_mode:
            if cmd_line_run:
                print('cmd line run, abort mounting gdrive')
                exit(-1)
            from google.colab import drive
            project_path = path.join(self.root_gdrive_path, self.gdrive_project_path)
            self.gdrive_project_path = path.join(self.root_gdrive_path, self.gdrive_project_path)
            drive.mount(self.root_gdrive_path)
            self.gdrive_mounted = True
        
            path_with_imports = path.join(self.root_gdrive_path, self.gdrive_project_path)
            print("Files in path", path_with_imports)
            !ls /content/drive/My\ Drive/pp/GSN/FaceGenerator
            if path_with_imports not in os.sys.path:
                os.sys.path.append(path_with_imports)
                
class DatasetCache:
    def __init__(self, use_manually_downloaded_dataset):
        self.path = ""
        self.batch_size = 0
        self._data = None
        # should it use dataset downloaded manually or the one downloaded by tfds
        self.use_manually_downloaded_dataset = use_manually_downloaded_dataset

    def load_data(self, settings):
        if self._data is not None and self.path == settings.dataset_path:
            self.batch_size = settings.batch_size
            return self.data
        else:
            print("downloading and loading data")
            if self.use_manually_downloaded_dataset:
                self.download_dataset(settings)
            self._data = load_dataset(
                settings.dataset_path,
                settings.image_size,
                use_manually_downloaded_dataset=self.use_manually_downloaded_dataset
            )
            self.data_path = settings.dataset_path
            self.batch_size = settings.batch_size
            return self.data
        
    @property
    def data(self):
        if self._data is None:
            return None
        else:
            return self._data.batch(self.batch_size)

    def download_dataset(self, settings):
        import dataset_helpers as ds_helpers
        '''Downloads data to dataset_path/dataset_name directory'''
        print('dataset download path is {}'.format(settings.download_path))
        ds_helpers.download_extract('celeba', settings.download_path)
        
class TensorboardManager():
    def __init__(self):
        self.log_path = ''
        self.train_summary_writer = None
        self.test_summary_writer = None
        
    def initialize(self, settings):
        should_be_updated = False
        if settings.collab_mode and self.log_path != "tensorboard_logs":
            should_be_updated = True
            self.log_path = "/content/drive/My Drive/pp/GSN/FaceGenerator/saved_state/run_img_align_celeba/tensorboard_logs"
        elif not settings.collab_mode and self.log_path != settings.tensorboard_log_dir:
            should_be_updated = True
            self.log_path = settings.tensorboard_log_dir
                
        if should_be_updated:
            self.train_summary_writer = tf.summary.create_file_writer(path.join(self.log_path, 'train'))
            self.test_summary_writer = tf.summary.create_file_writer(path.join(self.log_path, 'test'))
            print('Initialized tensorboard log dir with path', self.log_path)
            self.launch(settings.collab_mode)
        
    def launch(self, collab_mode):
        if collab_mode:
            if cmd_line_run:
                print('cmd line mode, abort launching tensorboard')
                returns
            %reload_ext tensorboard
            %tensorboard --logdir "/content/drive/My Drive/pp/GSN/FaceGenerator/saved_state/run_img_align_celeba/tensorboard_logs"
            from tensorboard import notebook
            notebook.list() # View open TensorBoard instances
        else:
            print('open tensorboard with command')
            print('tensorboard --logdir {}'.format(self.log_path))
            
    def save(self, run_type, data, description, timestamp, datatype, step):
        '''
        run_type: either `train` or `test`
        data: represents scalar, images or list of scalars
        description: should be of a length of data, i.e. if data is a scalar, description should be a string
        datatype: one of 'scalar', 'scalars', 'images'
        '''
        def _save(writer):
            with writer.as_default():
                if datatype == 'scalars':
                    for value, name in zip(data, description):
                        tf.summary.scalar('{}_{}'.format(name, timestamp), value, step=step)
                elif datatype == 'scalar':
                    tf.summary.scalar('{}_{}'.format(description, timestamp), data, step=step)
                elif datatype == 'images':
                    tf.summary.image('{}_{}'.format(description, timestamp), data, step=step)
                else:
                    print('unknown type', datatype)

        if run_type == 'train' and self.train_summary_writer:
            _save(self.train_summary_writer)
        elif run_type == 'test' and self.train_summary_writer:
            _save(self.test_summary_writer)
        else:
            print('unrecognized option `run_type` or selected writer', run_type,'is None')    
            
class Environment():
    def __init__(self, collab_mode):
        self.settings = Settings(collab_mode)
        self.models = dict()
        self.datasetCache = DatasetCache(use_manually_downloaded_dataset=False)
        self.checkpointManager = None
        self.tensorboard = None

In [ ]:
# test downloading and loading data

dataset = DatasetCache(use_manually_downloaded_dataset=False)
settings = Settings(collab_mode)
if collab_mode:
    settings.subdataset_dir = 'img_align_celeba'
dataset.load_data(settings)

if not collab_mode:
    display_image_from_dataset(dataset.data)

In [ ]:
def prepare_embeddings_from_batch(batch):
    imgs, attributes = batch
    embeddings = tf.stack(list(attributes.values()))
    embeddings = tf.transpose(embeddings)
    return imgs, embeddings
    
def describe_embedding(orig_attributes, example):
    image, embedding_vector = example
    # display image
    img_ = (image+1)/2
    plt.imshow(img_)
    # describe image
    for key, value in zip(orig_attributes.keys(), embedding_vector):
        print(key, ' '*(30-len(key)), value.numpy())

def test_creating_embedding(dataset):
    for batch in dataset.data.take(1): # take one batch
        # get imgs and embeddings from that batch
        imgs, embeddings = prepare_embeddings_from_batch(batch)
        # describe first image in the batch and display it
        describe_embedding(batch[1], (imgs[0], embeddings[0]) )
    
# imgs, embeddings = prepare_embeddings_from_batch(batch, settings)
test_creating_embedding(dataset)

In [ ]:
display_image_from_dataset(dataset.data)
if not collab_mode and not cmd_line_run:
    !ls "./datasets/celeb_a/img_align_celeba" | wc -l

Noise Generator

In [ ]:
class NoiseGenerator(tf.keras.layers.Layer):

    def __init__(self, distribution_size):
        super().__init__()
        self.distribution_size = distribution_size
        # self.data_distributions = self.add_weight(shape=(num_classes, distribution_size), trainable=True)
        # self.data_distributions = tf.tile(tf.range(0, num_classes, dtype=tf.float32)[:, tf.newaxis], [1, distribution_size])
        # TODO:

    def call(self, inputs):
        # dists = tf.nn.embedding_lookup(self.data_distributions, inputs)
        # dists += tf.random.uniform(tf.shape(dists), -0.35, 0.35)
        # return dists
        # TODO
        return tf.random.normal([tf.shape(inputs)[0], self.distribution_size, 3])
    
    def diverse_distributions_loss(self):
        # TODO
        return None
    
    
class StackGANNoiseGenerator(tf.keras.layers.Layer):
    def __init__(self, distribution_size=100):
        super().__init__()
        self.distribution_size = distribution_size

    def call(self, inputs):
        # batch_size x distribution_size
        return tf.random.normal([tf.shape(inputs)[0], self.distribution_size])

Loss functions


In [ ]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
kl = tf.keras.losses.KLDivergence()

def min_max_discriminator_loss(real_out, gen_out):
    real_loss = bce(tf.ones_like(real_out), real_out)
    gen_loss = bce(tf.zeros_like(gen_out), gen_out)
    return real_loss + gen_loss


def min_max_generator_loss(gen_out):
    return - min_max_discriminator_loss(tf.ones_like(gen_out), gen_out)


def w_discriminator_loss(real_out, gen_out):
    res = - (tf.reduce_mean(real_out) - tf.reduce_mean(gen_out))
    return res


def w_generator_loss(gen_out):
    return - tf.reduce_mean(gen_out)

def kl_generator_loss(real, gen):
    return kl(real, gen)

Layers for generator and discriminator from StackGAN

In [ ]:
class GLU(tf.keras.layers.Layer):
    '''
    Multiplies first half of the last dimension
    by value of the second half of the last dimension.
    Layer introduced in https://github.com/hanzhanggit/StackGAN-v2/blob/master/code/model.py#L47
    
    Returns the same tensor with its last dimension halved and multiplied by sigmoid function activation.
    '''
    def __init__(self, **kwargs):
        super(GLU, self).__init__(**kwargs)

    def call(self, data):
        rank = data.shape.rank
        tensor_shape = data.shape.as_list()
        last_dim_half = tensor_shape[-1]//2
        # automatic slicing is impossible since the first dimension is None (batch size)
        

        # begins_1, begins_2 = np.zeros((rank,), dtype=np.int32), np.zeros((rank,), dtype=np.int32)
        # begins_2[-1] = last_dim_half # set offset for the last dimension

        # sizes = [tensor_shape[i] for i in range(rank)]
        # sizes[-1] = last_dim_half # get half of elements from the last dimension

        # return tf.slice(data, begins_1, sizes) * tf.math.sigmoid(tf.slice(data, begins_2, sizes))
        
        
        if rank == 2:
            return data[:, :last_dim_half] * tf.math.sigmoid(data[:, last_dim_half:])
        elif rank == 3:
            return data[:, :, :last_dim_half] * tf.math.sigmoid(data[:, :, last_dim_half:])
        elif rank == 4:
            return data[:, :, :, :last_dim_half] * tf.math.sigmoid(data[:, :, :, last_dim_half:])
        elif rank == 5:
            return data[:, :, :, :, :last_dim_half] * tf.math.sigmoid(data[:, :, :, :, last_dim_half:])

class GeneratorResidualLayer(tf.keras.layers.Layer):
    def __init__(self, filters, index, **kwargs):
        super(GeneratorResidualLayer, self).__init__(**kwargs)
        
        self.block = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters*2, 3, 1, 'same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            GLU(),
            tf.keras.layers.Conv2D(filters, 3, 1, 'same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
        ], name='generator_residual_layer_{}'.format(index))

    def call(self, data):
        residual = data
        out = self.block(data)
        return tf.keras.layers.add([out, residual])
    
class GeneratorUpsampleLayer(tf.keras.layers.Layer):
    def __init__(self, filters, **kwargs):
        super(GeneratorUpsampleLayer, self).__init__(**kwargs)
        self.block = tf.keras.Sequential([
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(filters*2, 3, 1, 'same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            GLU()
        ], name='generator_upsample_layer')

    def call(self, data):
        return self.block(data)

Generator


In [ ]:
class GANNetwork(tf.keras.Model):
    def __init__(self, model_name="Network", **kwargs):
        self.is_conditional = False
        if 'is_conditional' in kwargs:
            self.is_conditional = kwargs['is_conditional']
            # remove that entry from kwargs
            # tf raises an exception when a keyword is not from allowed_kwargs set
            del kwargs['is_conditional']
        super().__init__(name=model_name, **kwargs)
        
        self.model = None
        

    def print_layers(self):
        print(self.model)
        for layer in self.model:
            print(layer.name, ":", layer.input_shape, "->", layer.output_shape)

    def summary(self):
        self.model.summary()
        
    @property
    def output_layer(self):
        return self.model.get_layer(index=-1)
    
    @property
    def output_shape(self):
        return self.output_layer.output_shape[1:]

    @tf.function
    def call(self, data, training):
        return self.model(data)

    def save_model(self, save_path):
        if not path.exists(save_path):
            os.makedirs(save_path)
        filename = path.join(save_path, '{}_{}'.format(self.name, get_time()))
        print("Saving model", self.name, "as", filename)
        self.model.save(filename)
        
    def compile_model(self, optimizer, loss):
        self.model.compile(optimizer=optimizer, loss=loss)

In [ ]:
# variables from pytorch
init_w = 4 # ?
init_h = 4 #?
gen_init_filters = 64 # how many filters*2 (channels) should be in the first layer of the generator, n_g number in the paper
df_dim = 64
gf_dim=gen_init_filters # ?
z_dim=(100, 100) # size of the vector with normal distribution noise
embeddings_shape = (100, 40)
n_g = gen_init_filters
n_d = 64
batch_size=24
settings.batch_size = batch_size

In [ ]:
class GeneratorInitStage(GANNetwork):
    #input_shape = cfg.GAN.Z_DIM + cfg.GAN.EMBEDDING_DIM
    # or 
    # input_shape = cfg.GAN.Z_DIM if without conditioning
    def __init__(self, noise_shape, embeddings_shape, filters, model_name="GeneratorInitStage", **kwargs):
        # filters should be `gen_init_filters`
        super().__init__(model_name, **kwargs)
        self.initial_factor = 16
        self.filters = self.initial_factor*filters
        # if is conditional then the second dimension is has size of concatenated noise and embedding
        
        self.in_shape = \
            (noise_shape[1] + embeddings_shape[1]) if self.is_conditional else noise_shape[1]
#         print('in shape', self.in_shape)
        
        self.model = tf.keras.Sequential([
            tf.keras.layers.Dense(init_w*init_h*self.filters*2, input_shape=(self.in_shape, )),
            tf.keras.layers.BatchNormalization(),
            GLU(),
            tf.keras.layers.Reshape([init_w, init_h, self.filters]),
            GeneratorUpsampleLayer(self.filters//2),
            GeneratorUpsampleLayer(self.filters//4),
            GeneratorUpsampleLayer(self.filters//8),
            GeneratorUpsampleLayer(self.filters//16)
        ], name='generator_intermediate_init_output')
        
        self.output_model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(3, 3, 1, 'same', input_shape=self.output_shape)
        ], name = 'generator_init_stage_output')
        
    @tf.function
    def call(self, data, training):
        intermediate_output = self.model(data)
        output = self.output_model(intermediate_output)
        return intermediate_output, output

In [ ]:
class GeneratorNextStage(GANNetwork):
    @staticmethod
    def prepare_embedding_tensor(embedding, output_shape):
        embedding_shape = embedding.shape.as_list()
        # multiply output_shape[0] * output_shape[1]
        repeat_factor = tf.math.reduce_prod(output_shape)
        res = tf.keras.backend.repeat(embedding, repeat_factor)
        reshape_s = [embedding_shape[0], *output_shape, embedding_shape[-1]]
        print(res.shape.as_list(), reshape_s)
        # reshape to [batch_size, output_shape[0], output_shape[1], embedding_size]
        return tf.reshape(res, reshape_s)
    
    @staticmethod
    def concat_embedding_and_previous_output(embedding, previous_output):
        repeat_factor = 1
#         print(embedding.shape.as_list(), previous_output.shape.as_list())
        if previous_output.shape.rank == 4:
            # get second and third dimension
            previous_output_shape = tf.slice(previous_output.shape, begin=[1], size=[2]).numpy()
        else:
            raise("Rank of the previous_output is not equal to 4")
        embedding = GeneratorNextStage.prepare_embedding_tensor(embedding, previous_output_shape)
        print('embedding', embedding.shape.as_list())
        return tf.concat([embedding, previous_output], axis=-1)
    

    def __init__(self, embeddings_shape, number_of_residuals, previous_stage_shape, filters, model_name="GeneratorInitStage", **kwargs):
        super().__init__(model_name, **kwargs)
        
        # size should be equal to
        # (init_stage_output_y, init_stage_output_x, init_stage_output_channels
        self.in_shape = previous_stage_shape
        if self.is_conditional:
            # concat embeddings with channel
            self.in_shape = (self.in_shape[0], self.in_shape[1], self.in_shape[2] + embeddings_shape[1])
        
        self.model = tf.keras.Sequential()
        
        self.model.add(tf.keras.layers.Dense(filters*2, input_shape=self.in_shape))
        self.model.add(tf.keras.layers.BatchNormalization())
        self.model.add(GLU())
        
        self.model.add(GeneratorResidualLayer(filters, index=0))
        for i in range(number_of_residuals-1):
            self.model.add(GeneratorResidualLayer(filters, i+1))
        
        self.model.add(GeneratorUpsampleLayer(filters//2))
        
        self.output_model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(3, 3, 1, 'same', input_shape=self.output_shape)
        ], name = 'generator_next_stage_output')
        
    @tf.function
    def call(self, data, training):
        intermediate_output = self.model(data)
        output = self.output_model(intermediate_output)
        return intermediate_output, output

In [18]:
# test init and next stages
init_stage = GeneratorInitStage(noise_shape=z_dim,
                                embeddings_shape=embeddings_shape,
                                filters=gen_init_filters,
                                is_conditional=True)
# init_stage.summary()
# init_stage.output_model.summary()

h_code_shape = init_stage.output_shape # shape of the output of the INIT_STAGE_G intermediate_output layer

first_next_stage = GeneratorNextStage(
    embeddings_shape=embeddings_shape,
    number_of_residuals=2,
    previous_stage_shape=h_code_shape,
    filters=gf_dim,
    is_conditional=True)
# first_next_stage.summary()
# first_next_stage.output_model.summary()

# second_next_stage = GeneratorNextStage(
#     embeddings_shape=embeddings_shape,
#     number_of_residuals=2,
#     previous_stage_shape=first_next_stage.output_shape,
#     filters=gf_dim,
#     is_conditional=True)

# second_next_stage.summary()
# second_next_stage.output_model.summary()

noise_generator = StackGANNoiseGenerator()
for batch in dataset.data.take(1):
    imgs, embedding = prepare_embeddings_from_batch(batch)
    emb = tf.cast(embedding, dtype=tf.float32)
    
    noise = noise_generator(imgs)
    
    embeddings = tf.concat([emb, noise], axis=1)

    intermediate_out, init_out = init_stage(embeddings, training=False)
    print('intermediate out', tf.shape(intermediate_out).numpy())
    next_input = GeneratorNextStage.concat_embedding_and_previous_output(embedding=emb, previous_output=intermediate_out)
    out_next = first_next_stage(next_input, training=False)

intermediate out [10 64 64 64]
[10, 4096, 40] [10, 64, 64, 40]
embedding [10, 64, 64, 40]


In [ ]:
class Generator(GANNetwork):
    def __init__(self, input_shape, model_name="Generator", **kwargs):
        super().__init__(model_name, **kwargs)
        self.model = tf.keras.Sequential([
            # flat
            tf.keras.layers.Dense(1024, input_shape=input_shape),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Reshape([16, 16, 256]),
            # conv without stride (16x16)
            tf.keras.layers.Conv2D(256, 5, 1, 'same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            # t_conv with stride (32x32)
            tf.keras.layers.Conv2DTranspose(128, 5, 2, 'same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            # conv without stride (32x32)
            tf.keras.layers.Conv2D(64, 5, 1, 'same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            # t_conv with stride (64x64)
            tf.keras.layers.Conv2DTranspose(32, 5, 2, 'same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            # conv without stride
            tf.keras.layers.Conv2D(3, (1, 1), 1, 'same')
        ])

Discriminator


In [ ]:
class DiscriminatorEncodeImageLayer(tf.keras.layers.Layer):
    def __init__(self, filters, **kwargs):
        super(DiscriminatorEncodeImageLayer, self).__init__(**kwargs)
        self.block = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters, 4, 2, 'same', use_bias=False),
            tf.keras.layers.LeakyReLU(alpha=.2),
            
            tf.keras.layers.Conv2D(filters*2, 4, 2, 'same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=.2),
            
            tf.keras.layers.Conv2D(filters*4, 4, 2, 'same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=.2),
            
            
            tf.keras.layers.Conv2D(filters*8, 4, 2, 'same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=.2),
        ], name='discriminator_encoding_layer')

    def call(self, data):
        return self.block(data)

In [ ]:
class StackGANDiscriminator(GANNetwork):
    def __init__(self, input_shape, embedding_shape, filters, model_name="StackGANDiscriminator", **kwargs):
        super().__init__(model_name, **kwargs)
        
        self.filters = filters
        self.model = tf.keras.Sequential()
        self.model.add(DiscriminatorEncodeImageLayer(filters = self.filters))
        
        if self.is_conditional:
            self.model.add(tf.keras.layers.Reshape()
        
discriminator_64 = StackGANDiscriminator(input_shape=(64, 64, 3), filters=df_dim)

In [ ]:
class Discriminator(GANNetwork):

    def __init__(self, input_shape, model_name="Discriminator", **kwargs):
        super().__init__(model_name, **kwargs)

        # since discriminator is for classification it should be robust, thus, add
        # additional regularization like dropout to prevent from pixel attacks
        self.model = tf.keras.Sequential([
            # conv with stride (32x32)
            tf.keras.layers.Conv2D(64, 5, 2, 'same', input_shape=input_shape),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dropout(0.3),
            # conv with stride (16x16x128)
            tf.keras.layers.Conv2D(128, 3, 2, 'same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dropout(0.3),
            # flatten + hidden layer
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dropout(0.3),
            # prediction (LOGITS!)
            tf.keras.layers.Dense(1)
        ])

In [ ]:
generator = Generator(input_shape=settings.generator_input_shape)
generator.build((None, *settings.generator_input_shape))
generator.summary()

discriminator = Discriminator(input_shape=settings.image_shape)
discriminator.build(input_shape=(None, *settings.image_shape))
discriminator.summary()

First implementation, no stack gan
Generator:

   * Total params: 2,724,291
   * Trainable params: 2,721,283
   * Non-trainable params: 3,008
        
Discriminator:

   * Total params: 2,177,025
   * Trainable params: 2,176,513
   * Non-trainable params: 512

Training step



In [ ]:
def train_step_template(generator, discriminator, noise, d_optim, g_optim, d_loss_f, g_loss_f, kl_loss_f):

    @tf.function
    def _train_step_template(images, epoch):
        with tf.GradientTape() as d_tape, tf.GradientTape() as g_tape:
            real_out = discriminator(images, True)
            generated = generator(noise(images), True)
            gen_out = discriminator(generated, True)

            d_loss = d_loss_f(real_out, gen_out)
            g_loss = g_loss_f(gen_out)
            g_kl_loss = kl_loss_f(images, generated)

        d_grads = d_tape.gradient(d_loss, discriminator.trainable_variables)
        g_grads = g_tape.gradient(g_loss, generator.trainable_variables + noise.trainable_variables)

        d_optim.apply_gradients(zip(d_grads, discriminator.trainable_variables))
        g_optim.apply_gradients(zip(g_grads, generator.trainable_variables + noise.trainable_variables))
        
        return d_loss, g_loss, g_kl_loss

    return _train_step_template


Inference step



In [ ]:
def gen_step_template(generator, noise):
    @tf.function
    def _gen_step_template(images):
        return tf.clip_by_value(generator(noise(images), False), -1, 1)

    return _gen_step_template

Training

In [ ]:
def train(train_step, gen_step, epochs, data, settings, ckptManager, tensorboard_manager, save_images=True, display_images=True):
    loss_descriptions = ['discriminator loss', 'generator loss', 'kl generator loss']
    timestamp = get_time()
    max_batch_iter = 0
    for epoch in range(epochs):
        batch_iter = 0
        epoch_start = time.time()
        for images_and_attributes in data:
            images = images_and_attributes[0]
            batch_iter+=1
            train_result = train_step(images, epoch)
            tensorboard_manager.save('train',
                                     [np.average(loss) for loss in train_result],
                                     loss_descriptions,
                                     timestamp,
                                     'scalars',
                                     epoch*max_batch_iter+batch_iter
                                    )
            if (batch_iter+1) % 50 == 0:
                print("Saving checkpoint, iter", batch_iter+1)
                ckptManager.save()
        # save the number of batches in one epoch
        if epoch == 0:
            max_batch_iter = batch_iter

        epoch_end = time.time()
        print('-'*30)
        print('Epoch {0}/{1}, duration {2}'.format(epoch+1, epochs, epoch_end-epoch_start))
        #if (epoch + 1) % 5 == 0 or epoch == epochs-1:
        print("Saving checkpoint, epoch", epoch+1)
        ckptManager.save()

        images_to_generate = [img[0] for img in data.take(1)][0].numpy() # take one batch from train_data
        generated = gen_step(images_to_generate)
        
        tensorboard_manager.save('train',
                                 generated,
                                 '{}_{}/{}'.format(settings.run_name, epoch+1, epochs),
                                 timestamp,
                                 'images',
                                 epoch
                                )
            
        show_images(generated, epoch, settings, save_images=save_images, display_images=display_images)
        print('+'*30)


In [ ]:
# check how images are displayed/saved
def test_image_generation(settings):
    save_images = True
    display_images = True
    batch_size = 10
    data = load_dataset(batch_size=settings.batch_size)
    
    generator_input_shape = (64, 3)
    generator = Generator(input_shape=generator_input_shape)
    noise = NoiseGenerator(64)
    
    gen_step = gen_step_template(
        generator=generator,
        noise=noise
    )
    images_to_generate = [img for img in data.take(1)][0].numpy() # take one batch from train_data
    generated = gen_step(images_to_generate)
    show_images(generated, -1, settings, save_images=save_images, display_images=display_images)
    
# test_image_generation(Settings(collab_mode))

Training with Wasserstein loss function

In [ ]:
def get_models(settings):
    generator = Generator(input_shape=settings.generator_input_shape)
    generator.build((None, *settings.generator_input_shape))
#     generator.summary()

    discriminator = Discriminator(input_shape=settings.image_shape)
    discriminator.build(input_shape=(None, *settings.image_shape))
#     discriminator.summary()
    
    return generator, discriminator

In [ ]:
env = Environment(collab_mode)
env.settings.epochs = 30
env.settings.batch_size = 500
env.settings.subdataset_dir='img_align_celeba'
env.settings.save_models = True
print(env.settings.get_base_path)
# load data
env.datasetCache.load_data(env.settings)
# set models
env.models['generator'], env.models['discriminator'] = get_models(env.settings)
# setup tensorboard
env.tensorboard = TensorboardManager()
env.tensorboard.initialize(env.settings)

noise = StackGANNoiseGenerator(100)
d_optim = tf.keras.optimizers.Adam(1e-4)
g_optim = tf.keras.optimizers.Adam(1e-4)

checkpoint = tf.train.Checkpoint(generator_optimizer=g_optim,
                                 discriminator_optimizer=d_optim,
                                 generator=env.models['generator'],
                                 discriminator=env.models['discriminator'])
env.checkpointManager = tf.train.CheckpointManager(checkpoint=checkpoint,
                                                   directory=env.settings.checkpoint_dir,
                                                   max_to_keep=3
                                                  )
if env.checkpointManager.latest_checkpoint:
    print("restoring state from", env.checkpointManager.latest_checkpoint)
    checkpoint\
        .restore(env.checkpointManager.latest_checkpoint)
    
train_step = train_step_template(
    generator=env.models['generator'],
    discriminator=env.models['discriminator'],
    noise=noise,
    d_optim=d_optim,
    g_optim=g_optim,
    d_loss_f=w_discriminator_loss,
    g_loss_f=w_generator_loss,
    kl_loss_f=kl_generator_loss
)

gen_step = gen_step_template(
    generator=env.models['generator'],
    noise=noise
)

In [ ]:
print("Start time", get_time())
print('%'*30)
start = time.time()

train(
    train_step=train_step,
    gen_step=gen_step,
    epochs=env.settings.epochs,
    data=env.datasetCache.data,
    settings=env.settings,
    ckptManager=env.checkpointManager,
    tensorboard_manager=env.tensorboard
)

end = time.time()
print('%'*30)
print("End time", get_time())
print("seconds elapsed", end - start)

if env.settings.save_models:
    print('saving models')
    for model in env.models:
        env.models[model].save_model(env.settings.model_save_path)


uwagi:
* na początku generowane obrazki są białe, bardzo małe odchylenie w wartościach pikseli ok 17 dla skali 0-255
* generator używa tylko skali np 52-160
* później generator uczy się zwiększać odchylenie i wartości pikseli na obrazkach zwiększają się do przedziału 0-255
